# MACHINE LEARNING PARA CALCULAR A EVASÃO DOS ALUNOS DO PERÍODO PRESENCIAL

### IMPORTAÇÕES NESCESSÁRIAS

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

from lightgbm import LGBMClassifier
import pickle

from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
 


from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
from sklearn.model_selection import GridSearchCV


from sklearn.metrics import confusion_matrix
import seaborn as sns


### IMPORTANDO A TABELA BASE DO EXEL , COM OS DADOS QUE FORAM BUSCADOS, E UMA CONSULTA NO BANCO DE DADOS


### TRANSFORMA AS COLUNAS DE SIM E NÃO, EM BINÁRIAS, E AS DEMAIS COLUNAS EM FLOAT, TRANSFORMANDO OS VALORES NULOS PARA ZERO

In [ ]:
pd.set_option('future.no_silent_downcasting', True)

df                  = pd.read_excel('Base_dados.xlsx')
df['EVADIDO']       = df['EVADIDO'].apply(lambda valor: 1 if valor == 'S' else 0 )
df['DP']            = df['DP'].apply(lambda valor: 1 if valor == 'S' else 0 )
df['REP_FREQ']      = df['REP_FREQ'].apply(lambda valor: 1 if valor == 'S' else 0 )
df['INADIMPLENTE']  = df['INADIMPLENTE'].apply(lambda valor: 1 if valor == 'S' else 0 )


df['QTD_MESES_INADI'] = pd.to_numeric(df['QTD_MESES_INADI'], errors='coerce').fillna(0).astype(int)
df['QTD_DP'] = pd.to_numeric(df['QTD_DP'], errors='coerce').fillna(0).astype(int)
df['MEDIA_DP_POR_INADI'] = pd.to_numeric(df['MEDIA_DP_POR_INADI'], errors='coerce').fillna(0).astype(float)
df['MEDIA_COBRANCAS'] = pd.to_numeric(df['MEDIA_COBRANCAS'], errors='coerce').fillna(0).astype(float)
df['PERFIL_RISCO'] = pd.to_numeric(df['PERFIL_RISCO'], errors='coerce').fillna(0).astype(float)
df['QTD_PERIODOS'] = pd.to_numeric(df['QTD_PERIODOS'], errors='coerce').fillna(0).astype(float)
df['CURSO'] = pd.to_numeric(df['CURSO'], errors='coerce').fillna(0).astype(float)



### DECLARA AS COLUNAS DE BASE(X), E A DE SAÍDA (Y), OS EVADIDOS

In [ ]:
X = df[['CURSO', 'INADIMPLENTE', 'DP', 'QTD_DP', 'QTD_MESES_INADI', 'MEDIA_DP_POR_INADI','MEDIA_COBRANCAS','PERFIL_RISCO','QTD_PERIODOS']]
y = df['EVADIDO']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

### LIGHTGBM

### Criando modelo com uma pipeline de execução

Usei o ImbPipeline para deixar o trabalho mais organizado ao invés de ter diversas partes realizando todo processo. Desta forma eu consigo utilizar o smote (primeira parte), para fazer a sintetização de dados e deixa-los balanceados. O Scaler para organizar as informações em pesos e obter um melhor resultado no processo de treinamento e por fim criar o modelo e realizar todo o treinamento.

In [ ]:


# Pipeline com SMOTE + scaler + LGBM com pesos
lgb_pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=42)),
    ('scaler', StandardScaler()),
    ('clf', LGBMClassifier(
        objective='binary',
        metric='auc',
        n_estimators=1000,
        class_weight='balanced',  
        random_state=42
    )),
])


### Melhorando meu modelo

Utilizei o Grid Search CV para melhorar meu modelo com o apoio do Dandomized Search CV

In [ ]:
f1_scorer = make_scorer(f1_score, pos_label=1)
recall_scorer = make_scorer(recall_score, pos_label=1)
precision_scorer = make_scorer(precision_score, pos_label=1)


scoring = {
    'f1': make_scorer(f1_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
}


param_grid = {
    'clf__learning_rate': [0.01, 0.1],
    'clf__num_leaves': [31, 64],
    'clf__max_depth': [5, 10, -1],
}


grid = GridSearchCV(
    estimator=lgb_pipeline,
    param_grid=param_grid,
    scoring=scoring,
    refit='recall',  # foco em recall da evasão
    cv=5,
    n_jobs=-1,
    verbose=1
)


grid.fit(X_train,y_train)



y_probs = grid.predict_proba(X_test)[:, 1]
y_pred_thresh = (y_probs >= 0.4).astype(int)



print("Recall:", recall_score(y_test, y_pred_thresh))
print("F1 Score:", f1_score(y_test, y_pred_thresh))
print("Precision:", precision_score(y_test, y_pred_thresh))

### Analisando os resultados

In [ ]:

y_proba_lgb = grid.best_estimator_.predict_proba(X)[:, 1]


pr_table = []

print("\n### Resultados LightGBM com melhores params ###\n")
for thresh in [0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49]:
    y_pred = (y_proba_lgb > thresh).astype(int)
    print(f"--- Threshold: {thresh} ---")

    
    report = classification_report(y, y_pred)
    print(report)

    # Extrai precision e recall da classe 1 (3ª linha)
    linha_classe1 = report.split('\n')[3].split()
    precision = float(linha_classe1[1])
    recall = float(linha_classe1[2])

    # Adiciona dicionário à lista
    pr_table.append({
        'threshold': thresh,
        'precision': precision,
        'recall': recall
    })

# Converte para DataFrame
pr_table = pd.DataFrame(pr_table)

# Filtra e ordena os resultados relevantes
relevantes = pr_table[(pr_table['recall'] > 0.6) & (pr_table['precision'] > 0.5)]
relevantes = relevantes.sort_values(by='recall', ascending=False)

print(relevantes.head(10))


## MATRIZ DE CONFUSÃO PARA VERMOS OS RESULTADOS

In [ ]:
y_pred = (y_proba_lgb > 0.34).astype(int)
matriz = confusion_matrix(y, y_pred) #y (valores reais) e y_pred (valores previstos)
sns.heatmap(matriz, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Não Evadiu (prev)', 'Evadiu (prev)'],
            yticklabels=['Não Evadiu (real)', 'Evadiu (real)'])
plt.xlabel('Previsão')
plt.ylabel('Realidade')
plt.title('Matriz de Confusão 0.34')
plt.show()

In [ ]:
y_scores = y_proba_lgb  
 
precisions, recalls, thresholds = precision_recall_curve(y, y_scores)
 
plt.figure(figsize=(10, 6))
plt.plot(thresholds, precisions[:-1], 'b--', label='Precision')
plt.plot(thresholds, recalls[:-1], 'g-', label='Recall')
plt.xlabel('Threshold de corte')
plt.ylabel('Score')
plt.title('Precision vs Recall vs Threshold')
plt.legend(loc='best')
plt.grid()
plt.show()

## IMPORTANDO NOSSO MODELO PARA APLICAR DIA A DIA

In [ ]:
# Defina seu threshold
threshold = 0.34

# Empacote o modelo e o threshold em um dicionário
modelo_completo = {
    'modelo': grid,
    'threshold': threshold
}

# Salve 
with open('modelo_lgb_com_threshold_034_real.pkl', 'wb') as f:
    pickle.dump(modelo_completo, f)
